In [37]:
# https://www.wrighters.io/parameters-jupyter-notebooks-with-papermill/
import papermill as pm
from itertools import product

In [38]:
verbose = True  # True prints more output
# verbose = False  # True prints more output

# write run results to df_eval_results
# store_results = False
store_results = True

# number of the most-common symbols from days_lookbacks' performance rankings to keep
n_top_syms = 20  

# slice starts and ends for selecting the best performing symbols
syms_start = 0 
syms_end = 10 

fp_df_eval_results = 'Not_Applicable'
days_eval = 0  # no need to eval when getting the current picks    
n_samples = 1
run_type = 'current'


In [39]:
l_drop_last_n_rows = [0, 1, 2]

l_days_lookbacks = [
    [30, 60, 120],
    [15, 30, 60, 120],
    ]

In [40]:
run_params = dict(
                  # fp_df_eval_results=fp_df_eval_results,
                  # fp_df_picks=fp_df_picks,
                  verbose=verbose,
                  store_results=store_results,
                  n_top_syms=n_top_syms,
                  syms_start=syms_start,
                  syms_end=syms_end,
                  )

run_params

{'verbose': True,
 'store_results': True,
 'n_top_syms': 20,
 'syms_start': 0,
 'syms_end': 10}

In [41]:
items = list(product(l_days_lookbacks, l_drop_last_n_rows))

In [42]:
i = 1
for days_lookbacks, drop_last_n_rows in items:

  run_params['run_type'] = run_type
  run_params['fp_df_eval_results'] = fp_df_eval_results  
  run_params['n_samples'] = n_samples
  run_params['days_eval'] = days_eval
  run_params['days_lookbacks'] = days_lookbacks
  run_params['drop_last_n_rows'] = drop_last_n_rows

  print(f'{i:<4} of {len(items)}    run_type: {run_type:<10}    n_samples: {n_samples:<5.0f}    days_eval: {days_eval:<5.0f}    days_lookbacks: {str(days_lookbacks):<19}    drop_last_n_rows: {drop_last_n_rows}    fp_df_eval_results: {fp_df_eval_results:<26}')

  res = pm.execute_notebook(
    'yf_7a.ipynb',
    'yf_7aPM_out.ipynb',    
    parameters = run_params 
  )

  i += 1

1    of 6    run_type: current       n_samples: 1        days_eval: 0        days_lookbacks: [30, 60, 120]          drop_last_n_rows: 0    fp_df_eval_results: Not_Applicable            


Executing:   0%|          | 0/29 [00:00<?, ?cell/s]

PapermillExecutionError: 
---------------------------------------------------------------------------
Exception encountered at "In [15]":
---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
Cell In[15], line 91
     71 _sym_idx = ['SPY']
     72 df_SPY = df[iloc_start_eval:iloc_end_eval][_sym_idx]
     74 (
     75   _symbols,
     76   _period_yr,
     77   _retn,
     78   _DD,
     79   _UI,
     80   _MDD,
     81   _retnMean,
     82   _retnStd,
     83   _retnStd_div_UI,
     84   _CAGR,
     85   _CAGR_div_retnStd,
     86   _CAGR_div_UI,
     87   SPY_retnStd_d_UI,     
     88   SPY_CAGR,
     89   SPY_CAGR_d_retnStd,
     90   SPY_CAGR_d_UI,
---> 91 ) = symb_perf_stats_vectorized_v8(df_SPY)  
     93 print(f'\nSPY: retnStd/UI, CAGR/retnStd, CAGR/UI, CAGR: {SPY_retnStd_d_UI[0]:>17,.1f}, {SPY_CAGR_d_retnStd[0]:>17,.1f}, {SPY_CAGR_d_UI[0]:>17.1f}, {SPY_CAGR[0]:>17.1f}')
     95 # 

#### Check the run parameters before running. It will take about 1 hour and 20 minutes to run

In [ ]:
len_list = len(list(product(run_types, l_days_eval, l_days_lookbacks)))

i = 1
for run_type, days_eval, days_lookbacks in list(product(run_types, l_days_eval, l_days_lookbacks)):
  # use n_samples_train to scale n_samples of other run_types 
  if run_type == 'train':
    n_samples = n_samples_train  # train
    fp_df_eval_results = f'df_eval_results_train'    
  elif run_type == 'validate':
    n_samples = round(n_samples_train * 0.2/0.7)  # validate, proportion to .7,.2,.1 split of train, validate, test
    fp_df_eval_results = f'df_eval_results_validate'    
  elif run_type == 'test':
    n_samples = round(n_samples_train * 0.1/0.7)  # test, proportion to .7,.2,.1 split of train, validate, test  
    fp_df_eval_results = f'df_eval_results_test'
  elif run_type == 'current':
    fp_df_eval_results = 'Not_Applicable'
    days_eval = 0  # no need to eval when getting the current picks    
    n_samples = 1
    # fp_df_eval_results = f'df_eval_results_test'              
  else:
    n_samples = 2  # debug
    fp_df_eval_results = f'df_eval_results' 

  run_params['run_type'] = run_type
  run_params['fp_df_eval_results'] = fp_df_eval_results  
  run_params['n_samples'] = n_samples
  run_params['days_eval'] = days_eval
  run_params['days_lookbacks'] = days_lookbacks

  # print(f'run_type: {run_type:<10}    fp_df_eval_results: {fp_df_eval_results:<26}    n_samples: {n_samples:<5.0f}    days_eval: {days_eval:<5.0f}    days_lookbacks: {days_lookbacks}')
  print(f'{i:<4} of {len_list}    run_type: {run_type:<10}    fp_df_eval_results: {fp_df_eval_results:<26}    n_samples: {n_samples:<5.0f}    days_eval: {days_eval:<5.0f}    days_lookbacks: {days_lookbacks}')

  # res = pm.execute_notebook(
  #   'yf_7a_w.ipynb',
  #   'yf_7aPM_out.ipynb',    
  #   parameters = run_params 
  # )

  i += 1

#### Check run results

In [ ]:
from myUtils import pickle_load
path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# df = pickle_load(path_data_dump, 'df_eval_results_train')
# df = pickle_load(path_data_dump, 'df_eval_results_validate')
# df = pickle_load(path_data_dump, 'df_eval_results_test')
df = pickle_load(path_data_dump, 'df_picks')
df

In [ ]:
# tbl_concat.loc[tbl_concat['run_type'] == 'train']
df.loc[df['date_end_df_train'] == '2023-03-17', 'sym_freq_9']
# df.loc[df['date_end_df_train'] == '2023-03-17', 'sym_freq_8']